In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

## Read Data

In [12]:
csv_file='./data/players_%s.csv'
years = ['15', '16', '17', '18', '19', '20']

fifa_total = pd.DataFrame()
for year in years:
    fifa_data = pd.read_csv(csv_file % year)
    fifa_data['year'] = '20%s' % year
    fifa_total = fifa_total.append(fifa_data)
    
fifa_total.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,year
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,27,1987-06-24,169,67,Argentina,FC Barcelona,...,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,2015
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,29,1985-02-05,185,80,Portugal,Real Madrid,...,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,2015
2,9014,https://sofifa.com/player/9014/arjen-robben/15...,A. Robben,Arjen Robben,30,1984-01-23,180,80,Netherlands,FC Bayern München,...,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,2015
3,41236,https://sofifa.com/player/41236/zlatan-ibrahim...,Z. Ibrahimović,Zlatan Ibrahimović,32,1981-10-03,195,95,Sweden,Paris Saint-Germain,...,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,2015
4,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Neuer,28,1986-03-27,193,92,Germany,FC Bayern München,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015


### Define useful information (eg. players position group, useful columns)

In [13]:
forward = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw']
mid = ['lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'ldm', 'cdm', 'rdm']
back = ['lwb', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb']
keeper = ['gk']

In [14]:
player_info_col = ['sofifa_id', 'player_url', 'short_name', 'age', 'height_cm', 'weight_kg', 
                   'nationality', 'club', 'overall', 'potential', 'value_eur', 'wage_eur', 
                   'player_positions', 'preferred_foot']

In [15]:
player_abilities_col = ['pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 
                        'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 'gk_speed', 'gk_positioning', 
                        'player_traits', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 
                        'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy',
                        'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 
                        'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 
                        'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 
                        'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties', 
                        'mentality_composure', 'defending_marking', 'defending_standing_tackle', 'defending_sliding_tackle',
                        'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 'goalkeeping_positioning', 
                        'goalkeeping_reflexes']

### Handle Missing Data

In [16]:
fifa_total[player_abilities_col] = fifa_total[player_abilities_col].replace(np.NaN, 0)
fifa_total[player_abilities_col].head()

,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,93.0,89.0,86.0,96.0,27.0,63.0,0.0,0.0,0.0,0.0,...,76,0,25,21,20,6,11,15,14,8
1,93.0,93.0,81.0,91.0,32.0,79.0,0.0,0.0,0.0,0.0,...,85,0,22,31,23,7,11,15,14,11
2,93.0,86.0,83.0,92.0,32.0,64.0,0.0,0.0,0.0,0.0,...,80+1,0,29,26+2,26+1,10,8,11,5,15
3,76.0,91.0,81.0,86.0,34.0,86.0,0.0,0.0,0.0,0.0,...,91,0,25,41-8,27,13,15,10,9,12
4,0.0,0.0,0.0,0.0,0.0,0.0,87.0,85.0,92.0,86.0,...,37,0,25,25,25,87,85+3,92,90+6,86
